In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install accelerate
import torch
from torch.utils.data import Dataset,random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.2 MB/s 
     |████████████████████████████████| 182 kB 79.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 191 kB 7.3 MB/s 


In [ ]:
#this downloaded model is 1.3B model, we need 125M model only
# from transformers import GPTNeoForCausalLM, GPT2Tokenizer
# load_path = '/content/drive/MyDrive/prod_desc/gpt_neo_pretrained'
# model = GPTNeoForCausalLM.from_pretrained(load_path)
# tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")


In [ ]:
import os
path = '/content/drive/MyDrive/prod_desc/datasets/All_dataset.txt'
with open(path,'r') as file:
   data= file.read()

In [ ]:
list_data = data.split('###')

In [3]:
#loading the model
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

In [ ]:
max_length = max([len(tokenizer.encode(l)) for l in list_data])

In [ ]:
type(max_length)

int

In [ ]:
list_data[0]

'\nKeywords: Best HD quality videos, Best sound quality\nTitle: LG 55" LED HDTV\nProduct Description: The LG 55" LED HDTV offers a variety of innovative features that make watching television more enjoyable. In addition to the high quality display and sound, it supports Bluetooth and Wifi so that we can stream videos directly from the internet.\n'

In [ ]:
print(list_data[0])
print(tokenizer.encode(list_data[0]))
print(len(list_data[0]))
print(len(tokenizer.encode(list_data[0])))


Keywords: Best HD quality videos, Best sound quality
Title: LG 55" LED HDTV
Product Description: The LG 55" LED HDTV offers a variety of innovative features that make watching television more enjoyable. In addition to the high quality display and sound, it supports Bluetooth and Wifi so that we can stream videos directly from the internet.

[198, 9218, 10879, 25, 6705, 5572, 3081, 5861, 11, 6705, 2128, 3081, 198, 19160, 25, 17370, 5996, 1, 12365, 5572, 6849, 198, 15667, 12489, 25, 383, 17370, 5996, 1, 12365, 5572, 6849, 4394, 257, 4996, 286, 13097, 3033, 326, 787, 4964, 5581, 517, 20050, 13, 554, 3090, 284, 262, 1029, 3081, 3359, 290, 2128, 11, 340, 6971, 19263, 290, 370, 22238, 523, 326, 356, 460, 4269, 5861, 3264, 422, 262, 5230, 13, 198]
343
73


In [ ]:
# max_length = max([len(tokenizer.encode(description)) for description in descriptions])
max([len(tokenizer.encode())])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,txt_list,tokenizer,max_length):
      self.input_ids = []
      self.attn_masks = []
      self.labels=[]
      for txt in txt_list:
        encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>',truncation=True,max_length=max_length,padding='max_length')
        self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
        self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]



In [ ]:
encodings_dict

{'input_ids': [50257, 198, 9218, 10879, 25, 6705, 5572, 3081, 5861, 11, 6705, 2128, 3081, 198, 19160, 25, 17370, 5996, 1, 12365, 5572, 6849, 198, 15667, 12489, 25, 383, 17370, 5996, 1, 12365, 5572, 6849, 4394, 257, 4996, 286, 13097, 3033, 326, 787, 4964, 5581, 517, 20050, 13, 554, 3090, 284, 262, 1029, 3081, 3359, 290, 2128, 11, 340, 6971, 19263, 290, 370, 22238, 523, 326, 356, 460, 4269, 5861, 3264, 422, 262, 5230, 13, 198, 50256, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
dataset = CustomDataset(list_data,tokenizer,max_length)

In [ ]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
training_args = TrainingArguments(output_dir='./results', num_train_epochs=500, logging_steps=5000,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()


***** Running training *****
  Num examples = 87
  Num Epochs = 500
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 22000
  Number of trainable parameters = 125200128


Step,Training Loss
5000,0.059900
10000,0.037400
15000,0.035500
20000,0.034500




Training completed. Do not forget to share your model on huggingface.co/models =)




ValueError: ignored

## Output evaluation

In [ ]:
model.eval()

In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()

sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                max_length=300, top_p=0.95, temperature=0.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: 
Keywords: 6.1-inch display, AI & VR, AI & HMD
Title: iPhone 12 Pro
Product Description: The new iPhone 12 Pro is here and it's the best smartphone we've ever tested. With a 6.1-inch display, AI and VR, AI and HMD, the new iPhone 12 Pro is sure to put the icing on the cake - and will give you a leg up on your rivals with its stellar camera.

1: 
Keywords: Best in class image and sound quality
Title: Samsung 48-inch LED TV
Product Description: Looking for a great TV, but don't need an expensive one? The Samsung 48-inch LED TV is a great choice for any home theater setting. It offers full HD resolution, best in class picture and sound quality, making it a great choice for any home theater setup.

2: 
Keywords: Full HD resolution, Best in class picture and sound quality
Title: Samsung 32-inch LED TV
Product Description: Looking for a high-quality LED TV that won't break the bank? Look no further than the Samsung 32-inch LED TV. This TV offers full HD resolution, best in class picture a

In [ ]:
model.save_pretrained('/content/drive/MyDrive/prod_desc/')

Configuration saved in /content/drive/MyDrive/prod_desc/config.json
Model weights saved in /content/drive/MyDrive/prod_desc/pytorch_model.bin


In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained("/content/drive/MyDrive/prod_desc/Finetuned-own").cuda()
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()

sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                max_length=300, top_p=0.95, temperature=0.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
prod_name = "Puma sweatshirt"
short = "quick-dry, comfortable, athletic" 
n_examples = 5 #@param {type:"number"}

text = f"""Keywords: {short}
Title: {prod_name}
Product Description:"""

print(text)
input = tokenizer("<|startoftext|>"+text, return_tensors="pt").input_ids.cuda()


Keywords: quick-dry, comfortable, athletic
Title: Puma sweatshirt
Product Description:


In [ ]:
sample_outputs = model.generate(input, do_sample=True, top_k=50,
                                max_length=300, top_p=0.95, temperature=0.9, num_return_sequences=1)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Keywords: quick-dry, comfortable, athletic
Title: Puma sweatshirt
Product Description: Looking for a sweatshirt that's comfortable enough to wear to the gym and stylish enough to wear out? Look no further than our Quick Dry Sweatshirt. Made from breathable, quick-drying fabric, this sweatshirt will keep you feeling comfortable and looking great no matter where you wear it.



In [ ]:
sample_outputs = model.generate(input, do_sample=True, top_k=50,
                                max_length=100, top_p=0.95, temperature=0.7, num_return_sequences=1)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Keywords: quick-dry, comfortable, athletic
Title: Puma sweatshirt
Product Description: Looking for a sweatshirt that's comfortable enough to wear to the gym and stylish enough to wear out? Look no further than our Quick Dry Sweatshirt. Made from breathable, quick-drying fabric, this sweatshirt will keep you feeling comfortable and looking great no matter where you wear it.



In [ ]:
prod_name = "Nivea moisturising cream"
short = "no chemicals, sutiable for all types of skin " 

text = f"""Keywords: {short}
Title: {prod_name}
Product Description:"""

# print(text)
input = tokenizer("<|startoftext|>"+text, return_tensors="pt").input_ids.cuda()


sample_outputs = model.generate(input, do_sample=True, top_k=50,
                                max_length=100, top_p=0.95, temperature=0.7, num_return_sequences=1)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Keywords: no chemicals, sutiable for all types of skin 
Title: Nivea moisturising cream
Product Description: Nivea moisturising cream is a Niveaomniarilating, quick dry, comfortable all weather. Get that perfect fit in our Niveaomniarilating, quick dry, comfortable.



In [ ]:
prod_name = "Vivo X40"
short = "5.8-inch display,triple camera, fast-charge " 

text = f"""Keywords: {short}
Title: {prod_name}
Product Description:"""

# print(text)
input = tokenizer("<|startoftext|>"+text, return_tensors="pt").input_ids.cuda()


sample_outputs = model.generate(input, do_sample=True, top_k=50,
                                max_length=100, top_p=0.95, temperature=0.7, num_return_sequences=1)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Keywords: 5.8-inch display,triple camera, fast-charge 
Title: Vivo X40
Product Description: The Vivo X40 is the ultimate choice for those who want the best of everything. With a massive 5.8-inch display, dual camera, and the latest Snapdragon 845 processor, the V40 is sure to turn heads. And with Fastcharge technology, you'll be able to charge your phone in no time.



## Train with amz dataset

In [15]:
path = '/content/drive/MyDrive/prod_desc/datasets/train 1.txt'
with open(path) as f:
  dataset1 = f.read()

In [16]:
dataset1

'<|endoftext|>\nCategory: Electronics > Television & Video > Televisions > LED & LCD TVs\nShort description: B07FPP6TB5, 4.6 out of 5 stars     42,827 ratings 4.6 out of 5 stars, #6 in Amazon Devices & Accessories (See Top 100 in Amazon Devices & Accessories) #1 in LED & LCD TVs #1 in Smart TVs #5 in Streaming Media Players , September 24, 2018\nTitle: Insignia NS-32DF310NA19 32-inch Smart HD TV - Fire TV\nBulletpoints: \n- Insignia HD Smart TV – Fire TV delivers 720p picture quality with deep blacks and rich colors.\n- With the Fire TV experience built-in, enjoy tens of thousands of channels, apps, and Alexa skills, including Disney+, Netflix, YouTube, Prime Video, Hulu, SHOWTIME, STARZ, and more\n- Fire TV seamlessly integrates live over-the-air TV and streaming channels on a unified home screen (HD antenna required).\n- Easily control your TV with the included Voice Remote with Alexa—plus, launch apps, search for titles, play music, switch inputs, control smart home devices, and mor

In [9]:
len(dataset1.split('<|endoftext|>'))

4941

In [10]:
max_length = max([len(tokenizer.encode(l)) for l in dataset1.split('<|endoftext|>')])

In [11]:
max_length

1361

In [41]:
class AmzDataset(Dataset):
    def __init__(self,txt_list,tokenizer,max_length):
      self.input_ids = []
      self.attn_masks = []
      self.labels=[]
      for txt in txt_list:
        encodings_dict = tokenizer(txt+'<|endoftext|>' ,truncation=True,max_length=max_length,padding='max_length')
        self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
        self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [42]:
all_txt = dataset1.split('<|endoftext|>')
#removing all the empty spaces present in list
for x in all_txt:
  if x=='':
    all_txt.remove(x)
dataset = AmzDataset(all_txt,tokenizer,max_length)

In [ ]:
all_txt[:5]

In [32]:
input_ids = []
attn_masks = []
labels=[]
for i,txt in enumerate(all_txt):
  if i>1 and i<3:
    encodings_dict = tokenizer(txt+'<|endoftext|>' ,truncation=True,max_length=100,padding='max_length')
    input_ids.append(torch.tensor(encodings_dict['input_ids']))
    attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    

In [33]:
encodings_dict

{'input_ids': [50256, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258], 'attention_mask': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [43]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=5000,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [44]:
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2871
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 7180
  Number of trainable parameters = 125200128


Step,Training Loss


KeyboardInterrupt: ignored